## Defining and training multi-layer artificial neural networks in Keras

Fortunately, the past decade has resulted in a large number of tools to efficiently work with very large, so-called deep neural networks. Among the more prominent tools are <a href="https://en.wikipedia.org/wiki/Theano_(software)">theano</a> and <a href="https://en.wikipedia.org/wiki/TensorFlow">Tensorflow</a>, from the University of Montreal and Google Inc., respectively, and the <a href="https://en.wikipedia.org/wiki/Keras">Keras</a> library to conveniently interface with them. We will be using Keras in this class to study the most important deep neural network architectures. 

Keras provides a series of standard models, the simplest being a sequential model that allows the user to stack layers of artifical neurons together:

In [5]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(2,input_dim=3,kernel_initializer='random_uniform'))

This example creates a sequential neural network with one "dense" layer with 2 artificial neurons, input dimension of 3, and uniformly random initial weights. Keras offers the following options <code>random_uniform</code>, <code>random_normal</code>, and <code>zero</code>, which are self-explanatory. Using a <i>dense</i> layer means that all inputs to a layer are connected to every neuron, and all outputs of a layer connect to all neurons of the next layer. The multi-layer perceptron shown above is a dense network.